# World of Warcraft Player Stats

Pulling player stats from the World of Warcraft API to help Guilds and Raiding parties to figure out team structuures.



### [1] Run required packages

In [4]:
library(httr)
library(jsonlite)
library(dplyr)
library(tidyverse)

Warning message:
"package 'httr' was built under R version 3.6.3"Warning message:
"package 'dplyr' was built under R version 3.6.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'tidyverse' was built under R version 3.6.3"-- Attaching packages --------------------------------------- tidyverse 1.3.0 --
v ggplot2 3.3.3     v purrr   0.3.4
v tibble  3.1.0     v stringr 1.4.0
v tidyr   1.1.3     v forcats 0.4.0
v readr   1.3.1     
Warning message:
"package 'ggplot2' was built under R version 3.6.3"Warning message:
"package 'tibble' was built under R version 3.6.3"Warning message:
"package 'tidyr' was built under R version 3.6.3"Warning message:
"package 'purrr' was built under R version 3.6.3"-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter()  masks stats::filter()
x 

### [1] Function to get player data

In [131]:
#function to get player data

get.raider.stats <- function(Realm, Raider, access_token){

    #GET data from API
    profile_data = GET(paste0('https://eu.api.blizzard.com/profile/wow/character/',Realm,'/',Raider,'?namespace=profile-eu&locale=en_GB',access_token))
    data = fromJSON(rawToChar(profile_data$content))
    
    PVP = GET(paste0(data$pvp_summary$href,access_token))
    PVP_Stat = fromJSON(rawToChar(PVP$content))
    
    PVP_Last = GET(paste0(PVP_Stat$brackets$href[3],access_token))
    PVP_Last_Data = fromJSON(rawToChar(PVP_Last$content))
    
    
    #Collect required attributes
    GuildName <- data$guild$name
    Player <- data$name
    Race <- data$race$name
    CharacterClass <- data$character_class$name
    ActiveSpec <- data$active_spec$name
    ItemLevel <- data$average_item_level
    RBG_Rating <- PVP_Last_Data$rating
    RBG_Played <- PVP_Last_Data$season_match_statistics$played
    RBG_Won <- PVP_Last_Data$season_match_statistics$won
    RBG_Lost <- PVP_Last_Data$season_match_statistics$lost
    
    
    Output <- data.frame(GuildName,
                         Player,
                         Race,
                         CharacterClass,
                         ActiveSpec,
                         ItemLevel,
                         RBG_Rating,
                         RBG_Played,
                         RBG_Won,
                         RBG_Lost)
    
    return(Output)
}

### [2] User facing function to clean data and create table

In [180]:
rbg.stats <- function(raiders, realm, token) {
    
results <- list()

for (i in raiders){

results[[i]] <- get.raider.stats(realm,i,token)
    
    }

results <- do.call(rbind,results)

results$WinRate <- round(results$RBG_Won/results$RBG_Played*100,2)
results$Experience <- ifelse(results$RBG_Played > mean(results$RBG_Played),1,0)
results$WinningPlayer <- ifelse(results$WinRate > mean(results$WinRate),1,0)
results$HighRated <- ifelse(results$RBG_Rating > mean(results$RBG_Rating),1,0)
results$HighItemLevel <- ifelse(ItemLevel > mean(ItemLevel),1,0)

results$Score = rowSums(results[,c(12,13,14,15)])

results <- results %>% select(c(1:11,16))

return(results)
    
}

### [3] User inputs

In [181]:
access_token <- "&access_token=USAOiJfo0ORK42eNpH5WGhu3s2lGnHa1F9"

realm <- "kazzak"

raiders <- list('peakyskengz',
                'amphali',
                'pejn',
                'busko',
                'fionah',
                'heraphim')

rbg.stats(raiders, realm, access_token)

,GuildName,Player,Race,CharacterClass,ActiveSpec,ItemLevel,RBG_Rating,RBG_Played,RBG_Won,RBG_Lost,WinRate,Score
peakyskengz,Guilty Casual,Peakyskengz,Orc,Hunter,Marksmanship,225,1618,79,41,38,51.90,1
amphali,Synergize,Amphali,Blood Elf,Mage,Frost,224,1631,25,14,11,56.00,1
pejn,Synergize,Pejn,Blood Elf,Demon Hunter,Vengeance,225,1810,46,28,18,60.87,2
busko,Guilty Casual,Busko,Undead,Priest,Holy,214,1562,36,17,19,47.22,0
fionah,Guilty Casual,Fionah,Blood Elf,Monk,Windwalker,212,1627,39,22,17,56.41,1
heraphim,Guilty Casual,Heraphim,Highmountain Tauren,Druid,Balance,221,1818,131,66,65,50.38,2
